## Importing Libraries and Initial Setup

The following libraries and modules are imported and initialized for text preprocessing and feature extraction:

`pandas`: Library for data manipulation and analysis.

`string`: Contains common string operations and constants.

`nltk`: Natural Language Toolkit, used for tokenization and stop words.

`spacy`: Library for advanced natural language processing.

`sklearn.feature_extraction.text.TfidfVectorizer`: For converting text to TF-IDF features.

NLTK data (`punkt` and `stopwords`) is downloaded for tokenization and filtering out common stop words.

The `en_core_web_trf` model from spaCy is loaded for processing text with transformers.

In [ ]:
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')

nlp = spacy.load("en_core_web_trf")

## Filipino Stopwords

The following list contains commonly used Filipino stopwords. These are words that are typically filtered out in text processing as they do not carry significant meaning in the context of text analysis.

These stopwords include common words such as articles, prepositions, and conjunctions that are usually removed during text preprocessing to focus on more meaningful words in text analysis.

In [2]:
filipino_stopwords = [
    "a", "ako", "ang", "ano", "at", "ay", "ibang", "ito", "iyon", "ka",
    "kami", "kanila", "kanya", "kayo", "laki", "mga", "na", "ng", "ni",
    "nito", "nang", "sa", "sila", "tayo", "walang", "yung", "si", "bawat",
    "kung", "hindi", "para", "dahil", "doon", "baka", "kapag", "saan",
    "sino", "siya", "tama", "yan", "o", "pala", "pero", "wala", "huwag",
    "muna", "na", "naman", "pag", "sana", "tulad", "upang", "bago", 
    "dati", "iba", "madami", "nakita", "pagkatapos", "pati", "sabi", "sana"
]

## Function: `load_dataset`

The `load_dataset` function is used to load a dataset from a CSV file. It handles various potential errors during the file loading process.

### Code

In [3]:
def load_dataset(file_path):
    try:
        data = pd.read_csv(file_path)
        print("Dataset loaded successfully!")
        return data
    except FileNotFoundError:
        print("File not found. Please check the file path.")
    except pd.errors.EmptyDataError:
        print("File is empty. Please check the file content.")
    except pd.errors.ParserError:
        print("Error parsing file. Please check the file format.")
    except Exception as e:
        print(f"An error occurred: {e}")

## Loading Dataset and Extracting Sentences

The following code snippet demonstrates how to load a dataset from a CSV file and extract sentences from it.

### Code

In [ ]:
file_path = '../data/sample_dataset.csv'
data = load_dataset(file_path)

sentences = data['sentence'].tolist()

## Function: `format_list_as_string`

The `format_list_as_string` function converts a list of tokens into a formatted string representation.

### Code

In [5]:
def format_list_as_string(token_list):
    return str(token_list).replace("'", '"')

## Function: `print_table`

The `print_table` function displays a formatted table using the `rich` library, which provides enhanced terminal output for data visualization.

### Code

In [ ]:
def print_table(data, title="Table", num_samples=5):
    from rich.console import Console
    from rich.table import Table
    
    table = Table(title=title)
    
    for col in data.columns:
        table.add_column(col)

    for _, row in data.head(num_samples).iterrows():
        formatted_row = [format_list_as_string(row[col]) if isinstance(row[col], list) else row[col] for col in data.columns]
        table.add_row(*map(str, formatted_row))
    
    console = Console()
    console.print(table)

print_table(data, title="Original Data")

## Function: `convert_to_lowercase`

The `convert_to_lowercase` function converts all text in the 'sentence' column of a DataFrame to lowercase.

### Code

In [ ]:
def convert_to_lowercase(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].str.lower()
        
        data.to_csv(file_path, index=False)
        
        print("Sentence has been converted to lowercase.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

convert_to_lowercase(data)
print_table(data, title="Data After Lowercase Conversion")

## Function: `remove_punctuation`

The `remove_punctuation` function removes punctuation from all text in the 'sentence' column of a DataFrame.

### Code

In [ ]:
def remove_punctuation(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
        print("Punctuation has been removed.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

remove_punctuation(data)
print_table(data, title="Data After Punctuation Removal")



## Function: `remove_numbers`

The `remove_numbers` function removes numerical digits from all text in the 'sentence' column of a DataFrame.

### Code

In [ ]:
def remove_numbers(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].str.replace(r'\d+', '', regex=True)
        print("Numbers have been removed.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

remove_numbers(data)
print_table(data, title="Data After Numbers Removal")


## Function: `tokenize_sentences`

The `tokenize_sentences` function tokenizes each sentence in the 'sentence' column of a DataFrame into individual words.

### Code

In [ ]:
def tokenize_sentences(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda x: word_tokenize(x))
        print("Sentences have been tokenized.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

tokenize_sentences(data)
print_table(data, title="Data After Tokenization")

## Function: `remove_stopwords`

The `remove_stopwords` function removes stopwords from each tokenized sentence in the 'sentence' column of a DataFrame.

### Code

In [ ]:
def remove_stopwords(data):
    if 'sentence' in data.columns:
        english_stopwords = set(stopwords.words('english'))
        all_stopwords = english_stopwords.union(set(filipino_stopwords))
        
        data['sentence'] = data['sentence'].apply(lambda tokens: [word for word in tokens if word.lower() not in all_stopwords])
        print("Stopwords have been removed.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

remove_stopwords(data)
print_table(data, title="Data After Stopwords Removal")

## Function: `lemmatize_tokens`

The `lemmatize_tokens` function lemmatizes each token in the 'sentence' column of a DataFrame using spaCy's lemmatization.

### Code

In [ ]:
def lemmatize_tokens(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda tokens: [nlp(token)[0].lemma_ for token in tokens])
        print("Tokens have been lemmatized.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

lemmatize_tokens(data)
print_table(data, title="Data After Lemmatization")

## Function: `join_tokens`

The `join_tokens` function joins tokens in each list within the 'sentence' column of a DataFrame back into single sentences.

### Code

In [ ]:
def join_tokens(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda tokens: ' '.join(tokens))
        print("Tokens have been joined back into sentences.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

join_tokens(data)
print_table(data, title="Data After Joining Tokens")

## Function: `vectorize_with_tfidf`

The `vectorize_with_tfidf` function performs TF-IDF vectorization on the 'sentence' column of a DataFrame.

### Code

In [ ]:
def vectorize_with_tfidf(data):
    if 'sentence' in data.columns:
        vectorizer = TfidfVectorizer()

        tfidf_matrix = vectorizer.fit_transform(data['sentence'])

        print("TF-IDF Vectorization complete.")
        return tfidf_matrix, vectorizer
    else:
        print("Column 'sentence' not found in the DataFrame.")

tfidf_matrix, vectorizer = vectorize_with_tfidf(data)
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
tfidf_df['emotion'] = data['emotion'].values

print(tfidf_df.head())